In [ ]:
fig = px.box(taxo_count_extended , y=['Modification','Modification_Type'], x='Count',color='Modification_Type',
    labels={"Count":"OTU count", 'value':' Media modification'}, 
    color_discrete_sequence=muted_color_set,)

fig.update_layout(autosize=False,width=1100,height=1500,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.update_layout(title={'text': "What is the richness per donor in the different conditions based on OTUs ?",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'}, title_font_size=30)
fig.update_traces(orientation='h')
fig.show()

fig.write_image("OTU richness per sample for each donor_box.png", scale=10)

In [ ]:
from plotly.subplots import make_subplots
from skbio import write
from skbio import TreeNode
from skbio.diversity import alpha_diversity
from pandas.api.types import CategoricalDtype

base_tree = TreeNode.read('1-Alpha_diversity/Result/Datasets/mOTUsv2.5.treefile')

OTUs_df = pd.read_csv('0-Data_wrangling/taxo_v2_RA_full_df.csv', sep='\t')
OTU_ref = OTUs_df['OTU'].str.rsplit(' [', expand = True)
OTUs_df['Ref'] = OTU_ref[1].str.strip(']') 
OTUs_df = OTUs_df.drop(0)
OTUs_df['Ref']= OTUs_df['Ref'].str.replace('_',' ')

OTUs_df['OTU_reformated']= OTUs_df['OTU'].str.replace('[',' ')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.replace(']',' ')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.replace(':','-')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.split().agg(" ".join) 
OTUs_to_keep_in_tree = OTUs_df['Ref'].to_list()
OTU_dict = dict(zip(OTUs_df['Ref'], OTUs_df['OTU']))

my_tree = base_tree.shear(OTUs_to_keep_in_tree)

for node in my_tree.tips():
    for ref, replacement in OTU_dict.items():
        node.name = node.name.replace(ref, replacement)

taxo_v2_RA_full_df_T = taxo_v2_RA_full_df.T
taxo_v2_RA_full_df_T.drop(labels=['-1'], inplace=True, axis=1)

faith_pd_taxo = alpha_diversity('faith_pd', taxo_v2_RA_full_df_T, ids=taxo_v2_RA_full_df_T.index, otu_ids=taxo_v2_RA_full_df_T.columns, tree=my_tree)
faith_pd_taxo
faith_pd_taxo_df = {'Sample_ID': taxo_v2_RA_full_df_T.index, 'faith_pd_index': faith_pd_taxo}
faith_pd_taxo_df = pd.DataFrame(data=faith_pd_taxo_df)

faith_pd_taxo_df = pd.merge(left=faith_pd_taxo_df, right=metadata_df, how='left', on='Sample_ID')
faith_pd_taxo_df = faith_pd_taxo_df[['Sample_ID','faith_pd_index', 'Donor', 'Modification', 'Modification_Type']]

faith_pd_taxo_df = faith_pd_taxo_df[~faith_pd_taxo_df['Sample_ID'].str.contains('ISO|SCRN|BBE|MRS|NoAx')]

faith_pd_taxo_df.loc[faith_pd_taxo_df['Modification'] == 'Stool Sequencing', 'Modification_Type'] = "Stool Sequencing"
faith_pd_taxo_df['Modification_Type'] = faith_pd_taxo_df['Modification_Type'].replace(['[Antibiotics,Antibiotics]',
       '[Antibiotics,Bile acids,Antibiotics]', '[Antibiotics,Bile acids]',
       '[Antibiotics,Antibiotics,Antibiotics]'], 'Combinations')
faith_pd_taxo_df['Modification_Type'] = faith_pd_taxo_df['Modification_Type'].fillna('Totals')
faith_pd_taxo_df.sort_values(by=['Modification_Type','Modification'], inplace=True )


taxo_count = taxo_v2_RA_full_df_melted.copy()

taxo_count= taxo_count[~taxo_count['Sample_ID'].str.contains('ISO|pSCRN|MRS|BBE|NoAx')]
taxo_count = taxo_count[taxo_count["RA in percent"]!= 0]
taxo_count = taxo_count.groupby(['Sample_ID']).size()
taxo_count = pd.DataFrame(taxo_count)
taxo_count.reset_index(inplace=True)
taxo_count.columns=['Sample_ID','Count']
taxo_count = pd.merge(left=taxo_count, right=metadata_df, how='left', on='Sample_ID')
taxo_count = taxo_count[['Donor', 'Modification', 'Count']]
taxo_count = taxo_count.groupby(['Donor', 'Modification']).sum().reset_index()

for x in metadata_df['Donor'].unique():
    total_x = taxo_v2_RA_full_df_melted[taxo_v2_RA_full_df_melted['Sample_ID'].str.contains(x)]
    total_x = total_x[total_x["RA in percent"]!= 0]
    total_x = total_x['OTU'].nunique()
    total_OTU_x = {'Donor':x,'Count':total_x,'Modification':'Total on all media and stool'}
    taxo_count = taxo_count.append(total_OTU_x, ignore_index=True)

    total_x_minus_ctrl = taxo_v2_RA_full_df_melted[taxo_v2_RA_full_df_melted['Sample_ID'].str.contains(x)]
    total_x_minus_ctrl =total_x_minus_ctrl[~total_x_minus_ctrl['Sample_ID'].str.contains("CTRL")]
    total_x_minus_ctrl = total_x_minus_ctrl[total_x_minus_ctrl["RA in percent"]!= 0]
    total_x_minus_ctrl = total_x_minus_ctrl['OTU'].nunique()
    total_OTU_x_media = {'Donor':x,'Count':total_x_minus_ctrl,'Modification':'Total on all media only'}
    taxo_count = taxo_count.append(total_OTU_x_media, ignore_index=True) 

taxo_count = pd.pivot_table(taxo_count, values='Count', index='Donor', columns='Modification') #, fill_value=0
taxo_count = taxo_count.melt(ignore_index=False)
taxo_count.sort_values(by='Modification', inplace=True)
taxo_count.fillna(0, inplace=True)
taxo_count.reset_index(inplace=True)
taxo_count.columns=['Donor','Modification','Count']

metadata_benchmark_set = pd.read_csv('Data/Metadata_&_targets/Benchmark 2.0 set 50.csv', sep='\t')
taxo_count_extended = pd.merge(left=taxo_count, right=metadata_benchmark_set, how='left', on='Modification')
taxo_count_extended = taxo_count_extended[['Donor', 'Modification', 'Modification_Type','Count']]
taxo_count_extended.loc[taxo_count_extended['Modification'] == 'Stool Sequencing', 'Modification_Type'] = "Stool Sequencing"
taxo_count_extended['Modification_Type'] = taxo_count_extended['Modification_Type'].replace(['[Antibiotics,Antibiotics]',
       '[Antibiotics,Bile acids,Antibiotics]', '[Antibiotics,Bile acids]',
       '[Antibiotics,Antibiotics,Antibiotics]'], 'Combinations')
taxo_count_extended['Modification_Type'] = taxo_count_extended['Modification_Type'].fillna('Totals')


fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.03)
media_type_order = CategoricalDtype(['Amino acids', 'Antibiotics', 'Bile acids', 'Carbohydrates','Compound', 'Dilution', 'Incubation', 'SCFAs', 'pH','No_mod','Stool Sequencing','Totals'], ordered=True)
for mt in ['Amino acids', 'Antibiotics', 'Bile acids', 'Carbohydrates','Compound', 'Dilution', 'Incubation', 'SCFAs', 'pH','No_mod','Stool Sequencing',]:
    m_color= metadata_df[metadata_df['Modification_Type'] == mt]['Color_type'].values[0]
    df = faith_pd_taxo_df[faith_pd_taxo_df['Modification_Type'] == mt]
    fig.add_trace(go.Box(
    x = [df['Modification_Type'],df['Modification']],
    y = df['faith_pd_index'],
    name = mt,
    marker_color = m_color), row=2, col=1)

for mt in taxo_count_extended['Donor'].unique():
    df = taxo_count_extended[taxo_count_extended['Donor'] == mt]
    df['Modification_Type'] =df['Modification_Type'].astype(media_type_order)
    df.sort_values(by=['Modification_Type','Modification'], inplace=True )
    fig.add_trace(go.Scatter(
    x = [df['Modification_Type'],df['Modification']],
    y = df['Count'],
    mode='markers',#'lines',line_shape='spline',
    name = mt), row=1, col=1)
    
fig.update_layout(autosize=False,width=1300,height=800,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.update_xaxes(tickangle= -90) 

fig.update_layout(
    yaxis2=dict(
        titlefont_size=16,
        tickfont_size=12,
        title="Faith PD index",
        showspikes=True),
    yaxis=dict(
        titlefont_size=16,
        tickfont_size=12,
        title="OTU richness",
        showspikes=True),
    xaxis=dict(
        titlefont_size=16,
        tickfont_size=10,
        title="Media modifications",
        ticklabelposition= "outside bottom",
        dividerwidth=.5,
        showspikes=True),
    template='plotly_white',
)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.99,
    xanchor="right",
    x=1
),template='plotly_white',)

fig.show()
fig.write_image("OTU v2 richness and faith_pd diversity per sample.svg", scale=1)
fig.write_image("OTU v2 richness and faith_pd diversity per sample.png", scale=1)

In [ ]:
taxo_phylum_diversity = taxo_v2_RA_full_phylum_df_melted.copy()
taxo_phylum_diversity = pd.merge(left=taxo_phylum_diversity, right=metadata_df, how='left', on='Sample_ID')
taxo_phylum_diversity.fillna(0, inplace=True)
taxo_phylum_diversity = taxo_phylum_diversity[['OTU','Sample_ID','RA in percent', 'Donor','Media_Name', 'Modification','Modification_Type']]
taxo_phylum_diversity.sort_values(by=['Donor'], inplace=True ) 

heatmap_taxo_phylum_df = taxo_phylum_diversity.copy()#[taxo_family_RA_melted['Modification_Type'] == 'Stool Sequencing']
heatmap_taxo_phylum_df = heatmap_taxo_phylum_df[~heatmap_taxo_phylum_df['Sample_ID'].str.contains("CTRL|ISO|NoAx|SCRN|MRS|BBE")]
heatmap_taxo_phylum_df = heatmap_taxo_phylum_df[~heatmap_taxo_phylum_df['OTU'].str.contains("Bacteria phylum")]
heatmap_taxo_phylum_df_RA_mean = heatmap_taxo_phylum_df.groupby(['OTU','Modification']).median().reset_index()

heatmap_taxo_phylum_df_RA_mean['RA_range_number'] = [0 if i==0 else 1 if i<.1 else 2 if i<1 else 3 if i<10 else 4 if i<20 else 5 if i<50 else 6 for i in heatmap_taxo_phylum_df_RA_mean['RA in percent']]
heatmap_taxo_phylum_df_RA_mean = pd.pivot_table(data = heatmap_taxo_phylum_df_RA_mean,index=['OTU'], values='RA_range_number', columns='Modification', fill_value=0, sort=False)
heatmap_taxo_phylum_df_RA_mean = heatmap_taxo_phylum_df_RA_mean.loc[~(heatmap_taxo_phylum_df_RA_mean==0).all(axis=1)]
heatmap_taxo_phylum_df_RA_mean = heatmap_taxo_phylum_df_RA_mean.drop(labels=['-1'], axis=0)


heatmap_phylum_RA_mean= sns.clustermap(heatmap_taxo_phylum_df_RA_mean, 
                                   cmap=['#D5E2E6','#440154','#414487','#2a788e','#22a884','#7ad151','#fde725'],#"viridis", #ax = ax,
                                   cbar_kws={'label': 'Relative abundance level','shrink': .5},
                                   linewidths=.8,
                                   xticklabels=1,yticklabels=1,
                                   figsize=(18, 5), 
                                   cbar_pos=(.93, 0.87, 0.02, 0.2),
                        )

heatmap_phylum_RA_mean.fig.suptitle("Median relative abundance of phylum per media modification v2",fontsize=20, y=.90)
heatmap_phylum_RA_mean.ax_heatmap.collections[0].colorbar.set_ticks([0.4, 1.3, 2.1, 3, 3.8, 4.7, 5.5], size=0)
heatmap_phylum_RA_mean.ax_heatmap.collections[0].colorbar.set_ticklabels(['Not Detected', '0 to 0.1%', '0.1 to 1%','1 to 10%','10 to 20%','20 to 50%','Over 50%'], fontsize = 9)
plt.setp(heatmap_phylum_RA_mean.ax_heatmap.xaxis.get_majorticklabels(),rotation=45,ha="right", fontsize = 9)
plt.setp(heatmap_phylum_RA_mean.ax_heatmap.yaxis.get_majorticklabels(), rotation=0, size=9)
heatmap_phylum_RA_mean.ax_row_dendrogram.set_visible(False)
heatmap_phylum_RA_mean.ax_col_dendrogram.set_visible(False)

plt.savefig("Median relative abundance of phylum per media modification v2.jpg",dpi=500, bbox_inches = 'tight')
plt.savefig("Median relative abundance of phylum per media modification v2.svg",dpi=500, bbox_inches = 'tight')
plt.show()

In [ ]:
classification_df = pd.read_csv("0-Data_wrangling/classification_OTU_v2_reformated_nobracket.csv", sep= "\t", 
                                index_col=[0], names = ['OTU_ID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'species', 'OTU', 'IDcount'], skiprows=1)
classification_df.reset_index(drop=True, inplace=True)
classification_df.drop(labels=['Kingdom', 'Class', 'Order', 'Family', 'OTU_ID', 'IDcount', 'OTU'], inplace=True, axis=1)
classification_df.drop_duplicates(ignore_index=True, inplace=True)
classification_df.columns = ['Phylum','OTU']
classification_df

taxo_species_diversity = taxo_v2_RA_full_species_df_melted.copy()
taxo_species_diversity = pd.merge(left=taxo_species_diversity, right=metadata_df, how='left', on='Sample_ID')
taxo_species_diversity.fillna(0, inplace=True)
taxo_species_diversity = taxo_species_diversity[['OTU','Sample_ID','RA in percent', 'Donor','Media_Name', 'Modification','Modification_Type']]
taxo_species_diversity.sort_values(by=['Donor'], inplace=True ) 

heatmap_taxo_species_df = taxo_species_diversity.copy()#[taxo_family_RA_melted['Modification_Type'] == 'Stool Sequencing']
heatmap_taxo_species_df = heatmap_taxo_species_df[heatmap_taxo_species_df['Sample_ID'].str.contains("GAM_AN")]
heatmap_taxo_species_df = pd.merge(left=heatmap_taxo_species_df, right=classification_df, how='left', on='OTU')
heatmap_taxo_species_df['Phylum'].fillna(value='Unknown', inplace=True)
heatmap_taxo_species_df.columns = ['species', 'Sample_ID', 'RA in percent', 'Donor', 'Media_Name',
       'Modification', 'Modification_Type', 'Phylum']

heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df.copy()

heatmap_taxo_species_df_RA_Ratio['RA in percent']=heatmap_taxo_species_df_RA_Ratio['RA in percent'].replace(0, 0.001)

for donor in heatmap_taxo_species_df_RA_Ratio['Donor'].unique():
    df = heatmap_taxo_species_df_RA_Ratio[heatmap_taxo_species_df_RA_Ratio['Sample_ID'].str.contains(donor)]
    for species in df['species'].unique():
        subdf = df[df['species'] == species]
        gam_ra_row = subdf.loc[heatmap_taxo_species_df_RA_Ratio['Modification'] == 'Base']
        gam_ra_value = float(gam_ra_row['RA in percent'].values[0])
        subdf['RA_ratio'] = subdf['RA in percent'].divide(gam_ra_value)
        heatmap_taxo_species_df_RA_Ratio=heatmap_taxo_species_df_RA_Ratio.append(subdf, ignore_index=True)
        
heatmap_taxo_species_df_RA_Ratio.dropna(inplace=True)

heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df_RA_Ratio.groupby(['Phylum','species','Modification_Type','Modification']).mean('RA_ratio').reset_index()
heatmap_taxo_species_df_RA_Ratio['RA_ratio'] = heatmap_taxo_species_df_RA_Ratio['RA_ratio'].astype(int)
heatmap_taxo_species_df_RA_Ratio['RA_ratio'] = heatmap_taxo_species_df_RA_Ratio['RA_ratio'].replace(1, 0)

heatmap_taxo_species_df_RA_Ratio['RA_range_number'] = [1 if i<-2 else 
                                                     2 if i<-1 else 
                                                     3 if i<0 else 
                                                     4 if i==0 else 
                                                     5 if i<1 else 
                                                     6 if i<2 else 
                                                     7 for i in heatmap_taxo_species_df_RA_Ratio['RA_ratio']]
heatmap_taxo_species_df_RA_Ratio = pd.pivot_table(data = heatmap_taxo_species_df_RA_Ratio,index=['Phylum','species'], values='RA_ratio', columns=['Modification_Type','Modification'], fill_value=0, sort=False)
heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df_RA_Ratio.droplevel(0, axis=1)
heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df_RA_Ratio.droplevel(0, axis=0)
heatmap_taxo_species_df_RA_Ratio.drop(labels=['Base'], inplace=True, axis=1)
heatmap_taxo_species_df_RA_Ratio= heatmap_taxo_species_df_RA_Ratio.drop(labels=['-1'], axis=0)

heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df_RA_Ratio.loc[~(heatmap_taxo_species_df_RA_Ratio==4).all(axis=1)]

from random import randrange

heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df_RA_Ratio.loc[heatmap_taxo_species_df_RA_Ratio.index.str.contains('Lacto|Bifido')]
heatmap_taxo_species_df_RA_Ratio.columns = ['MRS','ISO-0019']
heatmap_taxo_species_df_RA_Ratio.sort_index(ascending=False ,inplace=True)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=False,
                    shared_yaxes=True, horizontal_spacing=0)

fig.append_trace(go.Bar(x=heatmap_taxo_species_df_RA_Ratio['MRS'],
                     y=heatmap_taxo_species_df_RA_Ratio.index, 
                     text=heatmap_taxo_species_df_RA_Ratio["MRS"].map('{:,.0f}'.format), #Display the numbers with thousands separators in hover-over tooltip 
                     textposition='inside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker_color='#4472c4'), 
                     1, 1) # 1,1 represents row 1 column 1 in the plot grid

fig.append_trace(go.Bar(x=heatmap_taxo_species_df_RA_Ratio['ISO-0019'],
                     y=heatmap_taxo_species_df_RA_Ratio.index, 
                     text=heatmap_taxo_species_df_RA_Ratio['ISO-0019'].map('{:,.0f}'.format),
                     textposition='inside',
                     orientation='h', 
                     width=0.7, 
                     showlegend=False, 
                     marker_color='#f58231'), 
                     1, 2) # 1,2 represents row 1 column 2 in the plot grid

fig.update_xaxes(title_text="MRS", row=1, col=1,type="log", range=[5,0])
fig.update_xaxes(title_text='ISO-0019', row=1, col=2,type="log", range=[0,5])

fig.update_layout(width=800, 
                  height=700,
                  title_x=0.9,
                  xaxis1={'side': 'top'},
                  xaxis2={'side': 'top'},
                  yaxis2={'side': 'left'},
                  template="plotly_white",)
fig.update_layout(showlegend=False, autosize=False,width=600,height=500,margin=dict(l=50,r=50,b=100,t=100,pad=4))
fig.write_image("MRS vs ISO-0019 for LABs.svg")
fig.show()

In [ ]:
classification_df = pd.read_csv('0-Data_wrangling/classification_OTU_v2_reformated.csv', sep='\t')
classification_df = classification_df[['OTU_ID','Phylum', 'Family', 'OTU']]

color_dict_phylum = {
    'Proteobacteria':'#aaffc3', 
    'Firmicutes':'#F28585',
    'Actinobacteria':'#dcbeff',
    'Verrucomicrobia':'#F2D377', 
    'Bacteroidetes':'#88CCEE',
    'Bacteria phylum [Proteobacteria/Bacteroidetes]':'#a9a9a9',
    'Bacteria phylum [Proteobacteria/Firmicutes]':'#a9a9a9',
    'Bacteria phylum [Proteobacteria/Actinobacteria]':'#a9a9a9',
    'Bacteria phylum incertae sedis':'#a9a9a9', 
    'Synergistetes':'#fabed4', 
    'Chloroflexi':'#bfef45',
    'Fusobacteria':'#8C7386', 
    'Lentisphaerae':'#bfef45', 
    'Euryarchaeota':'#9A6324'}

classification_df['Color_Phylum'] = classification_df['Phylum'].map(color_dict_phylum)
classification_df['Color_Phylum'].fillna('#a9a9a9', inplace=True)

classification_family_df= classification_df[['Family', 'Color_Phylum']]
classification_family_df['OTU']= classification_family_df['Family'].str.replace('[','')
classification_family_df['OTU']= classification_family_df['Family'].str.replace(']','')

color_dict_phylum_family = dict(zip(classification_family_df['Family'], classification_family_df['Color_Phylum']))
color_dict_phylum_family

In [ ]:
classification_df = pd.read_csv("0-Data_wrangling/classification_OTU_v2_reformated_nobracket.csv", sep= "\t", 
                                index_col=[0], names = ['OTU_ID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'OTU', 'IDcount'], skiprows=1)
classification_df.reset_index(drop=True, inplace=True)
classification_df= classification_df[['Phylum','Family']]
classification_df.drop_duplicates(ignore_index=True, inplace=True)
classification_df.columns = ['Phylum','OTU']


taxo_family_diversity = taxo_v2_RA_full_family_df_melted.copy()
taxo_family_diversity = pd.merge(left=taxo_family_diversity, right=metadata_df, how='left', on='Sample_ID')
taxo_family_diversity.fillna(0, inplace=True)
taxo_family_diversity = taxo_family_diversity[['OTU','Sample_ID','RA in percent', 'Donor','Media_Name', 'Modification','Modification_Type']]
taxo_family_diversity.sort_values(by=['Donor'], inplace=True ) 

heatmap_taxo_family_df = taxo_family_diversity.copy()#[taxo_family_RA_melted['Modification_Type'] == 'Stool Sequencing']
heatmap_taxo_family_df = heatmap_taxo_family_df[~heatmap_taxo_family_df['Sample_ID'].str.contains("CTRL|ISO|NoAx|SCRN|MRS|BBE")]
heatmap_taxo_family_df = pd.merge(left=heatmap_taxo_family_df, right=classification_df, how='left', on='OTU')
heatmap_taxo_family_df['Phylum'].fillna(value='Unknown', inplace=True)
heatmap_taxo_family_df.columns = ['family', 'Sample_ID', 'RA in percent', 'Donor', 'Media_Name',
       'Modification', 'Modification_Type', 'Phylum']

heatmap_taxo_family_df_RA_Ratio = heatmap_taxo_family_df.copy()

heatmap_taxo_family_df_RA_Ratio['RA in percent']=heatmap_taxo_family_df_RA_Ratio['RA in percent'].replace(0, 0.001)

for donor in heatmap_taxo_family_df_RA_Ratio['Donor'].unique():
    df = heatmap_taxo_family_df_RA_Ratio[heatmap_taxo_family_df_RA_Ratio['Sample_ID'].str.contains(donor)]
    for family in df['family'].unique():
        subdf = df[df['family'] == family]
        gam_ra_row = subdf.loc[heatmap_taxo_family_df_RA_Ratio['Modification'] == 'Base']
        gam_ra_value = float(gam_ra_row['RA in percent'].values[0])
        subdf['RA_ratio'] = np.log10(subdf['RA in percent'].divide(gam_ra_value))
        heatmap_taxo_family_df_RA_Ratio=heatmap_taxo_family_df_RA_Ratio.append(subdf, ignore_index=True)
        
heatmap_taxo_family_df_RA_Ratio.dropna(inplace=True)

heatmap_taxo_family_df_RA_Ratio = heatmap_taxo_family_df_RA_Ratio.groupby(['Phylum','family','Modification_Type','Modification']).mean().reset_index()
heatmap_taxo_family_df_RA_Ratio.sort_values(by=['Modification_Type','Modification'], inplace=True )


heatmap_taxo_family_df_RA_Ratio['RA_range_number'] = [1 if i<-2 else 
                                                     2 if i<-1 else 
                                                     3 if i<0 else 
                                                     4 if i==0 else 
                                                     5 if i<1 else 
                                                     6 if i<2 else 
                                                     7 for i in heatmap_taxo_family_df_RA_Ratio['RA_ratio']]
heatmap_taxo_family_df_RA_Ratio = pd.pivot_table(data = heatmap_taxo_family_df_RA_Ratio,index=['Phylum','family'], values='RA_range_number', columns=['Modification_Type','Modification'], fill_value=0, sort=False)
heatmap_taxo_family_df_RA_Ratio = heatmap_taxo_family_df_RA_Ratio.droplevel(0, axis=1)
heatmap_taxo_family_df_RA_Ratio = heatmap_taxo_family_df_RA_Ratio.droplevel(0, axis=0)
heatmap_taxo_family_df_RA_Ratio.drop(labels=['Base'], inplace=True, axis=1)
heatmap_taxo_family_df_RA_Ratio= heatmap_taxo_family_df_RA_Ratio.drop(labels=['-1'], axis=0)
heatmap_taxo_family_df_RA_Ratio.drop_duplicates(inplace=True)

heatmap_taxo_family_df_RA_Ratio = heatmap_taxo_family_df_RA_Ratio.loc[~(heatmap_taxo_family_df_RA_Ratio==4).all(axis=1)]
row_colors =heatmap_taxo_family_df_RA_Ratio.index.map(color_dict_phylum_family)
        
heatmap_family_RA_Ratio= sns.clustermap(heatmap_taxo_family_df_RA_Ratio, 
                                      col_cluster=False,
                                   cmap=['#440154','#364b9A','#4a7bb7','#D5E2E6','#22a884','#7ad151','#fde725'],#"viridis", #ax = ax,
                                   cbar_kws={'label': 'Log10 ratio of relative abundance level over GAM base','shrink': .5},
                                   linewidths=.8,
                                   xticklabels=1,yticklabels=1,
                                   figsize=(20, 15), 
                                   cbar_pos=(.93, 0.77, 0.02, 0.05),
                        )

for tick_label in heatmap_family_RA_Ratio.ax_heatmap.axes.get_yticklabels():
    tick_text = tick_label.get_text()
    tick_label.set_color(color_dict_phylum_family[tick_text])

heatmap_family_RA_Ratio.fig.suptitle("Mean relative abundance ratio over GAM-STD for family per media modification v2",fontsize=20, y=.90)
heatmap_family_RA_Ratio.ax_heatmap.collections[0].colorbar.set_ticks([1.4, 2.2, 3.1, 4, 4.8, 5.7, 6.5], size=0)
heatmap_family_RA_Ratio.ax_heatmap.collections[0].colorbar.set_ticklabels(
        ['Below log10= -2', 'log10= -1 to -2','log10= -1 to 0','Absent(log10=0)', 'log10= 0 to 1','log10= 1 to 2','Above log10= 2'], fontsize = 9)
plt.setp(heatmap_family_RA_Ratio.ax_heatmap.xaxis.get_majorticklabels(),rotation=45,ha="right", fontsize = 9)
plt.setp(heatmap_family_RA_Ratio.ax_heatmap.yaxis.get_majorticklabels(), rotation=0, size=9)
heatmap_family_RA_Ratio.ax_col_dendrogram.set_visible(False)


plt.savefig("Mean relative abundance ratio over GAM-STD for family per media modification v2.jpg",dpi=800, bbox_inches = 'tight')
plt.savefig("Mean relative abundance ratio over GAM-STD for family per media modification v2.svg",dpi=800, bbox_inches = 'tight')
plt.show()

In [ ]:
import ast

taxo_species_diversity = taxo_v2_RA_full_species_df_melted.copy()
taxo_species_diversity = pd.merge(left=taxo_species_diversity, right=metadata_df, how='left', on='Sample_ID')
taxo_species_diversity.fillna(0, inplace=True)
taxo_species_diversity = taxo_species_diversity[['OTU','Sample_ID','RA in percent', 'Donor','Media_Name', 'Modification','Modification_Type']]
taxo_species_diversity.sort_values(by=['Donor'], inplace=True ) 

heatmap_taxo_species_df = taxo_species_diversity.copy()#[taxo_family_RA_melted['Modification_Type'] == 'Stool Sequencing']
heatmap_taxo_species_df = heatmap_taxo_species_df[~heatmap_taxo_species_df['Sample_ID'].str.contains("CTRL|NoAx|SCRN|MRS|BBE")]
heatmap_taxo_species_df.columns = ['species', 'Sample_ID', 'RA in percent', 'Donor', 'Media_Name',
       'Modification', 'Modification_Type']

heatmap_taxo_species_df_RA_Ratio = heatmap_taxo_species_df.copy()
mod_presence=[]
for donor in heatmap_taxo_species_df_RA_Ratio['Donor'].unique():
    df = heatmap_taxo_species_df_RA_Ratio[heatmap_taxo_species_df_RA_Ratio['Sample_ID'].str.contains(donor)]
    for mod in df['Modification'].unique():
        if '[' in mod:
            mod_prime = ast.literal_eval(mod)
            mod_prime_df = df[df['Modification'].isin(mod_prime)]
            mod_df = df[df['Modification']==mod]
            mod_df = mod_df.append(mod_prime_df)
            pivot_mod_df = pd.pivot_table(data = mod_df,index=['species'], values='RA in percent', columns=['Modification'], fill_value=0, sort=False)
            pivot_mod_df= pivot_mod_df.drop(labels=['-1'], axis=0)
            pivot_mod_df[pivot_mod_df!=0] = 1
            pivot_mod_df = pivot_mod_df[pivot_mod_df[mod]==1]
            pivot_mod_df = pivot_mod_df.drop(labels=[mod], axis=1)
            pivot_mod_df['Sum'] = pivot_mod_df.sum(axis=1)
            n_mod = len(pivot_mod_df.columns)-1
            pivot_mod_df_values = pivot_mod_df['Sum'].value_counts()
            pivot_mod_df_values_proportion = (pivot_mod_df_values/len(pivot_mod_df.index)).round(3)
            pivot_mod_above_max_minus_1 = pivot_mod_df[pivot_mod_df['Sum']>=n_mod+1]
            pivot_mod_above_max_minus_1 = {'OTUs_above_max_minus_1':list(pivot_mod_above_max_minus_1.index)}
            result = { 'Modification': mod,'Number of modifications': n_mod,**pivot_mod_above_max_minus_1, **dict(pivot_mod_df_values_proportion)}
            mod_presence.append(result)

mod_presence_df = pd.DataFrame(mod_presence)
mod_presence_df.sort_index(inplace=True, axis=0)
mod_presence_df=mod_presence_df[['Modification','Number of modifications',0,1,2,3,4,5,6,'OTUs_above_max_minus_1']]
mod_presence_df.fillna(0, inplace=True)

mod_presence_df = mod_presence_df.groupby(by=['Number of modifications']).mean().unstack().reset_index()
mod_presence_df.columns = ['Present in x out of all conditions in the combination','Number of modifications','Proportion']
mod_presence_df['Present in x out of all conditions in the combination'] = mod_presence_df['Present in x out of all conditions in the combination'].astype(str)
fig = px.bar(mod_presence_df, x='Number of modifications', y='Proportion',color='Present in x out of all conditions in the combination',)
fig.update_layout(autosize=False,width=650,height=300,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.show()
fig.write_image("Modification presence in x out of all conditions in the combination - species.svg")

In [ ]:
import ast

function_kegg_diversity = pathways_KEGG_RA_df_melted_function.copy()
function_kegg_diversity.rename(columns={'Sample':'Sample_ID'}, inplace=True)
function_kegg_diversity = pd.merge(left=function_kegg_diversity, right=metadata_df, how='left', on='Sample_ID')
function_kegg_diversity.fillna(0, inplace=True)
function_kegg_diversity = function_kegg_diversity[['KO','Sample_ID','Count', 'Donor','Function ID', 'Modification','Modification_Type']]
function_kegg_diversity.sort_values(by=['Donor'], inplace=True ) 

heatmap_function_kegg_df = function_kegg_diversity.copy()#[taxo_family_RA_melted['Modification_Type'] == 'Stool Sequencing']
heatmap_function_kegg_df = heatmap_function_kegg_df[~heatmap_function_kegg_df['Sample_ID'].str.contains("CTRL|NoAx|SCRN|MRS|BBE")]
heatmap_function_kegg_df.columns = ['KO', 'Sample_ID', 'RA in percent', 'Donor', 'Function ID',
       'Modification', 'Modification_Type']

heatmap_function_kegg_df_RA_Ratio = heatmap_function_kegg_df.copy()
mod_presence=[]
for donor in heatmap_function_kegg_df_RA_Ratio['Donor'].unique():
    df = heatmap_function_kegg_df_RA_Ratio[heatmap_function_kegg_df_RA_Ratio['Sample_ID'].str.contains(donor)]
    for mod in df['Modification'].unique():
        if '[' in mod:
            mod_prime = ast.literal_eval(mod)
            mod_prime_df = df[df['Modification'].isin(mod_prime)]
            mod_df = df[df['Modification']==mod]
            mod_df = mod_df.append(mod_prime_df)
            pivot_mod_df = pd.pivot_table(data = mod_df,index=['KO'], values='RA in percent', columns=['Modification'], fill_value=0, sort=False)
            pivot_mod_df= pivot_mod_df.drop(labels=['-1'], axis=0)
            pivot_mod_df[pivot_mod_df!=0] = 1
            pivot_mod_df = pivot_mod_df[pivot_mod_df[mod]==1]
            pivot_mod_df = pivot_mod_df.drop(labels=[mod], axis=1)
            pivot_mod_df['Sum'] = pivot_mod_df.sum(axis=1)
            n_mod = len(pivot_mod_df.columns)-1
            pivot_mod_df_values = pivot_mod_df['Sum'].value_counts()
            pivot_mod_df_values_proportion = (pivot_mod_df_values/len(pivot_mod_df.index)).round(3)
            pivot_mod_above_max_minus_1 = pivot_mod_df[pivot_mod_df['Sum']>=n_mod+1]
            pivot_mod_above_max_minus_1 = {'OTUs_above_max_minus_1':list(pivot_mod_above_max_minus_1.index)}
            result = { 'Modification': mod,'Number of modifications': n_mod,**pivot_mod_above_max_minus_1, **dict(pivot_mod_df_values_proportion)}
            mod_presence.append(result)

mod_presence_df = pd.DataFrame(mod_presence)
mod_presence_df.sort_index(inplace=True, axis=0)
mod_presence_df=mod_presence_df[['Modification','Number of modifications',0,1,2,3,4,5,6,'OTUs_above_max_minus_1']]
mod_presence_df.fillna(0, inplace=True)

mod_presence_df = mod_presence_df.groupby(by=['Number of modifications']).mean().unstack().reset_index()
mod_presence_df.columns = ['Present in x out of all conditions in the combination','Number of modifications','Proportion']
mod_presence_df['Present in x out of all conditions in the combination'] = mod_presence_df['Present in x out of all conditions in the combination'].astype(str)
fig = px.bar(mod_presence_df, x='Number of modifications', y='Proportion',color='Present in x out of all conditions in the combination',)
fig.update_layout(autosize=False,width=650,height=300,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.show()
fig.write_image("Modification presence in x out of all conditions in the combination - function.svg")

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None,'display.max_colwidth', None)

color_dict_modif_type = dict(zip(metadata_df['Modification'], metadata_df['Color_type']))


taxa = 'Collinsella'
selected_taxa_heatmap = taxo_v2_RA_full_species_df.filter(like=taxa, axis=0).stack().reset_index()
selected_taxa_heatmap.columns = ['OTU','Sample_ID','RA in percent']
selected_taxa_heatmap = selected_taxa_heatmap[~selected_taxa_heatmap['Sample_ID'].str.contains("CTRL|NoAx|SCRN")]

selected_taxa_heatmap = selected_taxa_heatmap.groupby(by=['Sample_ID']).sum().unstack().reset_index()
selected_taxa_heatmap['Donor'] = selected_taxa_heatmap['Sample_ID'].map(donor)
selected_taxa_heatmap['Modification'] = selected_taxa_heatmap['Sample_ID'].map(modification)
selected_taxa_heatmap.columns = ['level_0','Sample_ID','RA in percent','Donor','Modification']
selected_taxa_heatmap = selected_taxa_heatmap[['Donor','Modification','RA in percent']]
selected_taxa_heatmap.fillna(0, inplace=True)

selected_taxa_heatmap['RA_range_number'] = [0 if i==0 else 1 if i<.1 else 2 if i<1 else 3 if i<10 else 4 if i<20 else 5 if i<50 else 6 for i in selected_taxa_heatmap['RA in percent']]
selected_taxa_heatmap_table = pd.pivot_table(selected_taxa_heatmap, values='RA in percent', index='Modification',columns='Donor')
selected_taxa_heatmap_table.fillna(0, inplace=True)
selected_taxa_heatmap_table = selected_taxa_heatmap_table.loc[~(selected_taxa_heatmap_table==0).all(axis=1)]
selected_taxa_heatmap_table['Index'] =selected_taxa_heatmap_table.index
selected_taxa_heatmap_table['Mean'] =selected_taxa_heatmap_table.mean(axis=1)
selected_taxa_heatmap_table = selected_taxa_heatmap_table[['Index','Mean']]

selected_taxa_heatmap_table.reset_index(inplace=True, drop=True)
selected_taxa_heatmap_table = selected_taxa_heatmap_table.iloc[[3,4,5,19,0,18,9,],:]

fig = px.bar(selected_taxa_heatmap_table, x='Index', y='Mean',color='Index',template='plotly_white',
             color_discrete_sequence=["#5ADB6E","#5ADB6E","#5ADB6E","#5ADB6E","#5ADB6E","#f58231","#f58231"],
             labels={"Index": "Modifications", "Mean": "RA in percent"}, opacity=.9) #color_discrete_map=color_dict_modif_type)

fig.update_layout(autosize=False,width=300,height=600,margin=dict(l=20,r=20,b=20,t=80,pad=4), showlegend=False)
fig.update_yaxes(title_text='',type="log", range=[-2,1])
fig.show()
fig.write_image("enrichment collinsella.svg")

In [ ]:
selected_taxa_heatmap_table_fig = sns.clustermap(selected_taxa_heatmap_table, 
                                      col_cluster=False,row_cluster=False,
                                   cmap=['#D5E2E6','#440154','#414487','#2a788e','#22a884','#7ad151','#fde725'],
                                   cbar_kws={'label': 'Relative abundance level','shrink': .5},
                                   linewidths=.8,
                                   xticklabels=1,yticklabels=1,
                                   figsize=(17, 17), 
                                   cbar_pos=(.93, 0.77, 0.02, 0.05),
                        )

selected_taxa_heatmap_table_fig.fig.suptitle(f"Heatmap of {taxa} per sample",fontsize=20, y=.90)
selected_taxa_heatmap_table_fig.ax_heatmap.collections[0].colorbar.set_ticks([0.4, 1.3, 2.1, 3, 3.8, 4.7, 5.5], size=0)
selected_taxa_heatmap_table_fig.ax_heatmap.collections[0].colorbar.set_ticklabels(['Not Detected', '0 to 0.1%', '0.1 to 1%','1 to 10%','10 to 20%','20 to 50%','Over 50%'], fontsize = 9)
plt.setp(selected_taxa_heatmap_table_fig.ax_heatmap.xaxis.get_majorticklabels(),rotation=45,ha="right", fontsize = 9)
plt.setp(selected_taxa_heatmap_table_fig.ax_heatmap.yaxis.get_majorticklabels(), rotation=0, size=9)
selected_taxa_heatmap_table_fig.ax_row_dendrogram.set_visible(False)
selected_taxa_heatmap_table_fig.ax_col_dendrogram.set_visible(False)
plt.savefig(f"Heatmap of {taxa} per sample.jpg",dpi=500, bbox_inches = 'tight')
plt.savefig(f"Heatmap of {taxa} per sample.svg",dpi=500, bbox_inches = 'tight')
plt.show()

In [ ]:
taxo_RA_per_media_sum_index = []
taxo_RA_per_media_sum = []
taxo_RA_per_media_sum_donor = []
for donor in metadata_df['Donor'].unique():
    taxo_RA_per_media_donor = taxo_v2_RA_full_df.drop(taxo_v2_RA_full_df.filter(regex='CTRL|ISO|pSCRN|MRS|BBE|NoAx').columns, axis=1)
    taxo_RA_per_media_donor = taxo_RA_per_media_donor.drop(labels=['-1'], axis=0)
    taxo_RA_per_media_donor = taxo_RA_per_media_donor.filter(regex=donor)
    taxo_RA_per_media_donor = taxo_RA_per_media_donor.loc[~(taxo_RA_per_media_donor ==0).all(axis=1)]
    taxo_RA_per_media_donor.fillna(0, inplace=True)
    if taxo_RA_per_media_donor.empty == False:
        taxo_RA_per_media_donor_sum = (taxo_RA_per_media_donor != 0).astype(int).sum(axis=1).value_counts(sort=False)
        taxo_RA_per_media_donor_sum.sort_index(inplace=True)
        if 10 in taxo_RA_per_media_donor_sum.index:
            taxo_RA_per_media_donor_sum.loc[10] = taxo_RA_per_media_donor_sum.loc[10:].sum()
            taxo_RA_per_media_donor_sum = taxo_RA_per_media_donor_sum.loc[1:10]
        else:
            closest_value = min(taxo_RA_per_media_donor_sum.index, key=lambda x:abs(x-10))
            taxo_RA_per_media_donor_sum.loc[10] = taxo_RA_per_media_donor_sum.loc[closest_value:].sum()
            taxo_RA_per_media_donor_sum.sort_index(inplace=True)
            taxo_RA_per_media_donor_sum = taxo_RA_per_media_donor_sum.loc[1:10]
        donor_list = [donor for d in range(0,len(taxo_RA_per_media_donor_sum))]
        taxo_RA_per_media_sum_index.append(list(taxo_RA_per_media_donor_sum.index))
        taxo_RA_per_media_sum.append(list(taxo_RA_per_media_donor_sum))
        taxo_RA_per_media_sum_donor.append(donor_list)

def flatten(t):
    return [item for sublist in t for item in sublist]    

taxo_RA_per_media_sum_index = flatten(taxo_RA_per_media_sum_index)
taxo_RA_per_media_sum = flatten(taxo_RA_per_media_sum)
taxo_RA_per_media_sum_donor = flatten(taxo_RA_per_media_sum_donor)

taxo_RA_per_media_sum= pd.DataFrame({'Number_of_media':taxo_RA_per_media_sum_index, 'Count':taxo_RA_per_media_sum, 'Donor':taxo_RA_per_media_sum_donor})
taxo_RA_per_media_sum.sort_values(by=['Donor','Number_of_media'], inplace=True)


fig = px.bar(taxo_RA_per_media_sum , y='Count', x='Number_of_media',barmode='group',
    labels={'Count':'Number of OTUs', "Number_of_media":"Number of media sustaining growth", '10':"10+"},
    color='Donor',template='plotly_white', ) 

fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.update_layout(title={'text': "What is the number of media allowing cultivation for each OTUs ?",'y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, title_font_size=30)


fig.write_image("Number of conditions allowing cultivation per OTU v2.png", scale=10)
fig.write_image("Number of conditions allowing cultivation per OTU v2.svg", scale=10)
fig.show()

In [ ]:
venn_taxo_df = taxo_v2_RA_full_df.copy()

OTU_in_seq_df = venn_taxo_df[venn_taxo_df.filter(regex="CTRL").columns] 
OTU_in_seq_df.fillna(0, inplace=True)
OTU_in_seq_df = OTU_in_seq_df.loc[~(OTU_in_seq_df==0).all(axis=1)]
OTU_set_in_seq= set(OTU_in_seq_df.index)
n_OTU_set_in_seq = len(OTU_set_in_seq)

OTU_in_benchmark_df = venn_taxo_df.drop(venn_taxo_df.filter(regex="CTRL|ISO|pSCRN|MRS|BBE|NoAx").columns, axis=1)
OTU_in_benchmark_df.fillna(0, inplace=True)
OTU_in_benchmark_df = OTU_in_benchmark_df.loc[~(OTU_in_benchmark_df==0).all(axis=1)]
OTU_set_in_benchmark= set(OTU_in_benchmark_df.index)
n_OTU_set_in_benchmark = len(OTU_set_in_benchmark)

OTU_in_enrichment_df = venn_taxo_df[venn_taxo_df.filter(regex="ISO").columns] 
OTU_in_enrichment_df.fillna(0, inplace=True)
OTU_in_enrichment_df = OTU_in_enrichment_df.loc[~(OTU_in_enrichment_df==0).all(axis=1)]
OTU_set_in_enrichment= set(OTU_in_enrichment_df.index)
n_OTU_set_in_enrichment = len(OTU_set_in_enrichment)

OTU_in_base_df = venn_taxo_df[venn_taxo_df.filter(regex="GAM_AN").columns]
OTU_in_base_df.fillna(0, inplace=True)
OTU_in_base_df = OTU_in_base_df.loc[~(OTU_in_base_df==0).all(axis=1)]
OTU_set_in_base= set(OTU_in_base_df.index)
n_OTU_set_in_base = len(OTU_set_in_base)

intersection_benchmark_enrichment = OTU_set_in_benchmark.intersection(OTU_set_in_enrichment)
n_intersection_benchmark_enrichment = len(intersection_benchmark_enrichment)

intersection_benchmark_ctrl = OTU_set_in_benchmark.intersection(OTU_set_in_seq)
n_intersection_benchmark_ctrl = len(intersection_benchmark_ctrl)

intersection_enrichment_ctrl = OTU_set_in_enrichment.intersection(OTU_set_in_seq)
n_intersection_enrichment_ctrl = len(intersection_enrichment_ctrl)

intersection_benchmark_enrichment_ctrl = OTU_set_in_enrichment.intersection(OTU_set_in_seq, OTU_set_in_benchmark)
n_intersection_benchmark_enrichment_ctrl = len(intersection_benchmark_enrichment_ctrl)

only_in_seq = OTU_set_in_seq.difference(OTU_set_in_benchmark,OTU_set_in_enrichment)
n_only_in_seq= len(only_in_seq)

only_in_benchmark = OTU_set_in_benchmark.difference(OTU_set_in_seq,OTU_set_in_enrichment)
n_only_in_benchmark= len(only_in_benchmark)

only_in_enrichment = OTU_set_in_enrichment.difference(OTU_set_in_seq,OTU_set_in_benchmark)
n_only_in_enrichment= len(only_in_enrichment)

intersection_base_ctrl = OTU_set_in_base.intersection(OTU_set_in_seq)
n_intersection_base_ctrl = len(intersection_base_ctrl)
only_in_base = OTU_set_in_base.difference(OTU_set_in_seq)
n_only_in_base = len(only_in_base)
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles

fig= plt.figure(figsize=(5, 5), dpi=500)

venn3(subsets = (n_only_in_seq, 
                 n_only_in_benchmark,
                 n_intersection_benchmark_ctrl,
                 n_only_in_enrichment,
                 n_intersection_enrichment_ctrl,
                 n_intersection_benchmark_enrichment,
                 n_intersection_benchmark_enrichment_ctrl), 
      set_colors=("#9470DB","#5ADB6E","#f58231"), alpha = 0.5)

plt.title('Venn diagram of the OTU presence in stool, benchmark and enrichment',fontname = 'Arial')
fig.tight_layout()
plt.savefig('Venn diagram of the OTU presence in stool, benchmark and enrichment.svg', transparent=True)

In [ ]:
only_in_seq = OTU_set_in_seq.difference(OTU_set_in_benchmark)
n_only_in_seq= len(only_in_seq)

only_in_benchmark_seq = OTU_set_in_benchmark.difference(OTU_set_in_seq)
n_only_in_benchmark_seq= len(only_in_benchmark_seq)

only_in_benchmark_enr = OTU_set_in_benchmark.difference(OTU_set_in_enrichment)
n_only_in_benchmark_enr= len(only_in_benchmark_enr)

only_in_enrichment = OTU_set_in_enrichment.difference(OTU_set_in_benchmark)
n_only_in_enrichment= len(only_in_enrichment)

fig= plt.figure(figsize=(5, 5), dpi=500)
venn2(subsets = (n_only_in_seq, 
                 n_only_in_benchmark_seq,
                 n_intersection_benchmark_ctrl), 
      set_colors=("#9470DB","#5ADB6E"), alpha = 0.5)

plt.title('Venn diagram of the OTU presence in stool benchmark',fontname = 'Arial')
fig.tight_layout()
plt.savefig('Venn diagram of the OTU presence in stool benchmark.svg', transparent=True)

fig= plt.figure(figsize=(5, 5), dpi=500)
venn2(subsets = (n_only_in_benchmark_enr,
                 n_only_in_enrichment,
                 n_intersection_benchmark_enrichment), 
      set_colors=("#5ADB6E","#f58231"), alpha = 0.5)

plt.title('Venn diagram of the OTU presence in benchmark and enrichment',fontname = 'Arial')
fig.tight_layout()
plt.savefig('Venn diagram of the OTU presence in benchmark and enrichment.svg', transparent=True)

In [ ]:
from skbio import write
from skbio import TreeNode
base_tree = TreeNode.read('1-Alpha_diversity/Result/Datasets/mOTUsv2.5.treefile')

OTUs_df = pd.read_csv('0-Data_wrangling/taxo_v2_RA_full_df.csv', sep='\t')
OTU_ref = OTUs_df['OTU'].str.rsplit(' [', expand = True)
OTUs_df['Ref'] = OTU_ref[1].str.strip(']') 
OTUs_df = OTUs_df.drop(0)
OTUs_df['Ref']= OTUs_df['Ref'].str.replace('_',' ')

OTUs_df['OTU_reformated']= OTUs_df['OTU'].str.replace('[',' ')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.replace(']',' ')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.replace(':','-')
OTUs_df['OTU_reformated']= OTUs_df['OTU_reformated'].str.split().agg(" ".join) 
OTUs_to_keep_in_tree = OTUs_df['Ref'].to_list()
OTU_dict = dict(zip(OTUs_df['Ref'], OTUs_df['OTU_reformated']))

my_tree = base_tree.shear(OTUs_to_keep_in_tree)

for node in my_tree.tips():
    for ref, replacement in OTU_dict.items():
        node.name = node.name.replace(ref, replacement)

export_tree = write(my_tree, format="newick", into='1-Alpha_diversity/phylogenetic_tree_targeted_enrichment.nwk')

with open('1-Alpha_diversity/phylogenetic_tree_targeted_enrichment.nwk', 'r') as file :
  filedata = file.read()
filedata = filedata.replace("'", '')
with open('1-Alpha_diversity/phylogenetic_tree_targeted_enrichment.nwk', 'w') as file:
  file.write(filedata)

In [ ]:
presence_stool_culture = OTUs_df.copy()
presence_stool_culture_OTUs = presence_stool_culture[['OTU_reformated','OTU']]

presence_stool = presence_stool_culture.filter(regex='CTRL', axis=1)
presence_stool['Presence_stool'] = presence_stool.sum(axis=1)
presence_stool['Presence_in_stool'] = presence_stool['Presence_stool'].apply(lambda x: 1 if x >0 else 0)

presence_culture_benchmark = presence_stool_culture.drop(presence_stool_culture.filter(regex='ISO|pSCRN|NoAx|MRS|BBE|CTRL').columns, axis=1)
presence_culture_benchmark['Presence_culture_benchmark'] = presence_culture_benchmark.sum(axis=1)
presence_culture_benchmark['Presence_in_culture_benchmark'] = presence_culture_benchmark['Presence_culture_benchmark'].apply(lambda x: 1 if x >0 else 0)

presence_culture_enrichment = presence_stool_culture.filter(regex='ISO', axis=1)
presence_culture_enrichment['Presence_culture_enrichment'] = presence_culture_enrichment.sum(axis=1)
presence_culture_enrichment['Presence_in_culture_enrichment'] = presence_culture_enrichment['Presence_culture_enrichment'].apply(lambda x: 1 if x >0 else 0)

presence_stool_culture_to_attach_to_tree = presence_stool_culture_OTUs.join(presence_stool['Presence_in_stool'])
presence_stool_culture_to_attach_to_tree = presence_stool_culture_to_attach_to_tree.join(presence_culture_benchmark['Presence_in_culture_benchmark'])
presence_stool_culture_to_attach_to_tree = presence_stool_culture_to_attach_to_tree.join(presence_culture_enrichment['Presence_in_culture_enrichment'])

presence_stool_culture_to_attach_to_tree = presence_stool_culture_to_attach_to_tree[['OTU_reformated', 'Presence_in_stool', 'Presence_in_culture_benchmark','Presence_in_culture_enrichment' ]]

presence_stool_culture_to_attach_to_tree.to_csv('presence_stool_culture_to_attach_to_tree.txt', index=False, header=False)

header = '''#label is used in the legend table (can be changed later)
DATASET_BINARY
#Each node can have multiple associated values, and each value will be represented by a symbol (defined in FIELD_SHAPES) with corresponding color and label (from FIELD_COLORS and FIELD_LABELS).
#Possible values (defined under DATA below) for each node are 1 (filled shapes), 0 (empty shapes) and -1 (completely omitted).


SEPARATOR COMMA

DATASET_LABEL,label1

COLOR,#ff0000

FIELD_SHAPES,1,1,1
FIELD_LABELS,Stool,Benchmark, Enrichment
FIELD_COLORS,#9470DB,#5ADB6E,#f58231
DATA

'''

f = open('presence_stool_culture_to_attach_to_tree.txt','r+')
lines = f.readlines() # read old content
f.seek(0) # go back to the beginning of the file
f.write(header) # write new content at the beginning
for line in lines: # write old content after new
    f.write(line)
f.close()

In [ ]:
classification_df = pd.read_csv('0-Data_wrangling/classification_OTU_v2_reformated.csv', sep='\t')
classification_df = classification_df[['OTU_ID','Kingdom','Phylum']]

color_dict_phylum = {
    'Proteobacteria':'#aaffc3', 
    'Firmicutes':'#F28585',
    'Actinobacteria':'#dcbeff',
    'Verrucomicrobia':'#F2D377', 
    'Bacteroidetes':'#88CCEE',
    'Bacteria phylum [Proteobacteria/Bacteroidetes]':'#a9a9a9',
    'Bacteria phylum [Proteobacteria/Firmicutes]':'#a9a9a9',
    'Bacteria phylum [Proteobacteria/Actinobacteria]':'#a9a9a9',
    'Bacteria phylum incertae sedis':'#a9a9a9', 
    'Synergistetes':'#fabed4', 
    'Chloroflexi':'#bfef45',
    'Fusobacteria':'#8C7386', 
    'Lentisphaerae':'#bfef45', 
    'Euryarchaeota':'#9A6324'}

taxo_color_to_attach_to_tree = OTUs_df[['OTU_reformated','OTU']]
otu_id = taxo_color_to_attach_to_tree['OTU'].str.rsplit(' [', expand = True)
OTU_ref = OTUs_df['OTU'].str.rsplit(' [', expand = True)
taxo_color_to_attach_to_tree['OTU_ID'] = OTU_ref[1].str.strip(']')

taxo_color_to_attach_to_tree = pd.merge(left= taxo_color_to_attach_to_tree, right=classification_df)
taxo_color_to_attach_to_tree['Color_Phylum'] = taxo_color_to_attach_to_tree['Phylum'].map(color_dict_phylum)
taxo_color_to_attach_to_tree['Range']="range"
taxo_color_to_attach_to_tree = taxo_color_to_attach_to_tree[['OTU_reformated', 'Range','Color_Phylum','Phylum']]

taxo_color_to_attach_to_tree.to_csv('taxo_color_to_attach_to_tree.txt', index=False, header=False)

header = '''TREE_COLORS
#use this template to define branch colors and styles, colored ranges and label colors/font styles/backgrounds


SEPARATOR COMMA

#First 3 fields define the node, type and color
#'range': defines a colored range (colored background for labels/clade)
#'clade': defines color/style for all branches in a clade
#'branch': defines color/style for a single branch
#'label': defines font color/style for the leaf label
#'label_background': defines the leaf label background color

#for 'range', field 4 defines the colored range label (used in the legend)

#for 'label', field 4 defines the font style ('normal',''bold', 'italic' or 'bold-italic') and field 5 defines the numeric scale factor for the font size (eg. with value 2, font size for that label will be 2x the standard size)
#for 'clade' and 'branch', field 4 defines the branch style ('normal' or 'dashed') and field 5 defines the branch width scale factor (eg. with value 0.5, branch width for that clade will be 0.5 the standard width)

DATA


'''

f = open('taxo_color_to_attach_to_tree.txt','r+')
lines = f.readlines() # read old content
f.seek(0) # go back to the beginning of the file
f.write(header) # write new content at the beginning
for line in lines: # write old content after new
    f.write(line)
f.close()

In [ ]:
import scipy.cluster.hierarchy as hrc
import scipy.spatial.distance as dist
import numpy.ma as ma

# Describe how OTUs cluster relative to their distance in growth preference
    # group by mean distance for Source target to have the average distance accross donors
all_level_growth_distance_mean = all_level_growth_distance_genus.groupby(['Source', 'Target']).mean("Distance").reset_index() 
all_level_growth_distance_mean = pd.pivot_table(all_level_growth_distance_mean, values="Distance", index='Source', columns='Target')

all_level_growth_distance_mean.to_csv('genus_growth_distance_hellinger_mean_donor.csv', sep="\t")

# remove nan and replace by value above the max distance for OTUs present in only few samples and not all

all_level_growth_distance_mean_np = np.where(np.isnan(all_level_growth_distance_mean), 
                                             ma.array(all_level_growth_distance_mean, 
                                                      mask=np.isnan(all_level_growth_distance_mean)).mean(axis=0), 
                                             all_level_growth_distance_mean)
#all_level_growth_distance_mean_np = np.nan_to_num(all_level_growth_distance_mean, nan=40)

    # get the condensed version of the array
condensed_all_levels = dist.squareform(all_level_growth_distance_mean_np, checks=False)


plt.figure(figsize=(5,25))
hrc.set_link_color_palette(muted_color_set)
linkage_method = "average"
linkage_matrix = hrc.linkage(condensed_all_levels, method=linkage_method)

# calculate the cophenetic correlation coeficient to find the ocrrelation between all pairwise distances and the result in the clustering
cophenet_dist = hrc.cophenet(linkage_matrix, condensed_all_levels)
cophenet_dist = round(cophenet_dist[0],3)

hrc.dendrogram(linkage_matrix, 
               labels=all_level_growth_distance_mean.index, 
               orientation="left", leaf_font_size=9,show_contracted=True,
                )

ax=plt.gca()
ylbls = ax.get_ymajorticklabels()
for lbl in ylbls:
    lbl.set_color(color_dict_phylum_genus[lbl.get_text()])
    
plt.title(f"Dendrogram of the growth distances between distance all genus using the {linkage_method} linkage method (Cophenet = {cophenet_dist})")
plt.savefig(f"dendrogram_distance_all_genus_average_donors_{linkage_method}.svg" ,dpi=500, bbox_inches = 'tight',transparent=True)
plt.show()

In [ ]:
# all Sources against all Sources distance per donor comparaison
from sklearn.metrics.pairwise import pairwise_distances

def growth_profile_distance_for_level(df):
    source_data =df.copy()
    source_data= source_data.drop(source_data.filter(regex='ISO|pSCRN|MRS|BBE|NoAx').columns, axis=1)
    final_df = []
    # Split taxo df of different level per donor since 
    # we want to comapre over the different media,  and not including all samples in the dataset 
    for donor in metadata_df['Donor'].unique():
        donor_source_data_df = source_data.filter(like=donor)
        # drop seq, iso and noax samples
        donor_source_data_df = donor_source_data_df.drop(donor_source_data_df.filter(regex='CTRL|NoAx|ISO').columns, axis=1)
        # change names to modification type
        donor_source_data_df.rename(columns=modification, inplace=True)
        # keep only rows present in culture for that donor and transpose to get the Sources in columns
        donor_source_data_df= donor_source_data_df[~(donor_source_data_df == 0).all(1)] 
            
        donor_source_data_df = donor_source_data_df
        # calculate distance between Sources in the donor for the different media
        hellinger_dist = pairwise_distances(((donor_source_data_df.T.values)**(1/2)), metric = "euclidean")
        #hellinger_dist_pvalues =hellinger_dist.copy()
        #hellinger_dist = np.triu(hellinger_dist) 
        hellinger_dist = pd.DataFrame(hellinger_dist, index=donor_source_data_df.columns, 
                                        columns=donor_source_data_df.columns)
        ## Melt for a resulting DF being : Donor, Taxa1, Taxa2, distance
        hellinger_dist_melted = hellinger_dist.melt(ignore_index=False)
        hellinger_dist_melted.columns= ['Target', "Distance"]
        hellinger_dist_melted.reset_index(inplace=True)
        hellinger_dist_melted.columns= ["Source",'Target', "Distance"]
        hellinger_dist_melted["Donor"]=donor
        # Remove brackets and -1 comparaison
        hellinger_dist_melted['Source'] = hellinger_dist_melted['Source'].apply(lambda x: x.replace('[','').replace(']',''))
        hellinger_dist_melted['Target'] = hellinger_dist_melted['Target'].apply(lambda x: x.replace('[','').replace(']',''))
        hellinger_dist_melted = hellinger_dist_melted[hellinger_dist_melted["Source"]!= "-1"]
        hellinger_dist_melted = hellinger_dist_melted[hellinger_dist_melted["Target"]!= "-1"]
        #hellinger_dist_melted = hellinger_dist_melted[hellinger_dist_melted["Distance"]!= 0]
        # Stack DF for all donor to have the distribution of distances
        final_df.append(hellinger_dist_melted)
                
                #get the pvalue for the metric calculation using two tailed wilcoxon rank sum on matrix g2f

    final_df= pd.concat(final_df, ignore_index=True)
    final_df.sort_values(by=["Donor","Distance"], inplace=True)
    return final_df

all_level_growth_distance_modif_species = growth_profile_distance_for_level(taxo_v2_RA_full_species_df)
all_level_growth_distance_modif_species["Taxo_level"]= "Species"

all_level_growth_distance_modif_phylum = growth_profile_distance_for_level(taxo_v2_RA_full_phylum_df)
all_level_growth_distance_modif_phylum["Taxo_level"]= "Phylum"

all_level_growth_distance_modif_family = growth_profile_distance_for_level(taxo_v2_RA_full_family_df)
all_level_growth_distance_modif_family["Taxo_level"]= "Family"

all_level_growth_distance_modif_genus = growth_profile_distance_for_level(taxo_v2_RA_full_genus_df)
all_level_growth_distance_modif_genus["Taxo_level"]= "Genus"

all_level_growth_distance_modif_df = pd.concat([all_level_growth_distance_modif_phylum,
                                          all_level_growth_distance_modif_family,
                                          all_level_growth_distance_modif_genus, 
                                          all_level_growth_distance_modif_species
                                          ], ignore_index=True)

color_dict_modif_type = dict(zip(metadata_df['Modification'], metadata_df['Color_type']))

import scipy.cluster.hierarchy as hrc
import scipy.spatial.distance as dist
import numpy.ma as ma

# Describe how OTUs cluster relative to their distance in growth preference
    # group by mean distance for Source target to have the average distance accross donors
all_level_growth_distance_mean = all_level_growth_distance_modif_df.groupby(['Source', 'Target']).mean("Distance").reset_index() 
all_level_growth_distance_mean = pd.pivot_table(all_level_growth_distance_mean, values="Distance", index='Source', columns='Target')
all_level_growth_distance_mean_np = np.where(np.isnan(all_level_growth_distance_mean), 
                                             ma.array(all_level_growth_distance_mean, 
                                                      mask=np.isnan(all_level_growth_distance_mean)).mean(axis=0), 
                                             all_level_growth_distance_mean)

    # get the condensed version of the array
condensed_all_levels = dist.squareform(all_level_growth_distance_mean_np, checks=False)

plt.figure(figsize=(3,15))
hrc.set_link_color_palette(muted_color_set)
linkage_method = "average"
linkage_matrix = hrc.linkage(condensed_all_levels, method=linkage_method)

# calculate the cophenetic correlation coeficient to find the ocrrelation between all pairwise distances and the result in the clustering
cophenet_dist = hrc.cophenet(linkage_matrix, condensed_all_levels)
cophenet_dist = round(cophenet_dist[0],3)

hrc.dendrogram(linkage_matrix, 
               labels=all_level_growth_distance_mean.index, 
               orientation="left", leaf_font_size=12,show_contracted=True
                )

ax=plt.gca()
ylbls = ax.get_ymajorticklabels()
for lbl in ylbls:
    lbl.set_color(color_dict_modif_type[lbl.get_text()])
    
plt.title(f"Dendrogram of the growth distances between all conditions based on all taxo levels using the {linkage_method} linkage method (Cophenet = {cophenet_dist})")
plt.savefig(f"dendrogram_distance_all_taxo_level_average_donors_{linkage_method}_modif.svg" ,dpi=500, bbox_inches = 'tight',transparent=True)
plt.show()

In [ ]:
fig = go.Figure()
for mt in all_level_growth_distance_df['Taxo_level'].unique():
    df = all_level_growth_distance_df[all_level_growth_distance_df['Taxo_level'] == mt]
    count_obs= df['Distance'].size
    fig.add_trace(go.Box(
    x = df["Taxo_level"],
    y = df['Distance'],
    boxpoints='all',jitter=0.5, pointpos=.5,#fillcolor='rgba(0,0,0,0)',
    marker_size=2,
    name = mt + " (n="+ str(count_obs)+ ")"
    ))

fig.update_layout(autosize=False,width=700,height=500,margin=dict(l=20,r=20,b=20,t=80,pad=4))
fig.update_xaxes(tickangle= -90) 

fig.update_layout(
    yaxis=dict(
        titlefont_size=16,
        tickfont_size=12,
        title="Hellinger distance (Euclidean distance calculated on the square root of relative abudnance)",
        showspikes=True),
    xaxis=dict(
        titlefont_size=16,
        tickfont_size=12,
        title="Taxonomic level",
        ticklabelposition= "outside bottom",
        dividerwidth=.5,
        showspikes=True),
    template='plotly_white',
    #barmode='stack',
)

fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=0.96,
    xanchor="right",
    x=1
))
fig.update_layout(title={'text': "What is the growth distance distribution per donor at taxonomic levels ?",'y':0.98,'x':0.5,'xanchor': 'center','yanchor': 'top'}, title_font_size=30)

fig.show()
fig.write_image("growth distance at different taxo levels per donor.svg", scale=1)
fig.write_image("growth distance at different taxo levels per donor.png", scale=1)